### Rag Pipeline data ingestion to vector db pipeline

In [87]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [88]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: test2.pdf
  ✓ Loaded 11 pages

Total documents loaded: 11


In [89]:
all_pdf_documents

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2018-03-05T09:43:57+01:00', 'author': 'agimeno', 'moddate': '2018-03-12T10:24:10-04:00', 'source': '..\\data\\pdf\\test2.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1', 'source_file': 'test2.pdf', 'file_type': 'pdf'}, page_content="The EUROCALL Review, Volume 25, No. 2, September 2017 \n \n 18 \nResearch paper \n \nA look at advanced learners’ use of mobile devices for \nEnglish language study: Insights from interview data \nMariusz Kruk \nUniversity of Zielona Gora, Poland \n______________________________________________________________ \nmkruk @ uz.zgora.pl \n  \nAbstract \nThe paper discusses the results of a study which explored advanced learners of English \nengagement with their mobile devices to develop learning experiences that meet their \nneeds and goals as foreign language learners. The data were collected from 20 students \nby means of a semi -structured interv

In [90]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [91]:
chunks=split_documents(all_pdf_documents)
chunks

Split 11 documents into 56 chunks

Example chunk:
Content: The EUROCALL Review, Volume 25, No. 2, September 2017 
 
 18 
Research paper 
 
A look at advanced learners’ use of mobile devices for 
English language study: Insights from interview data 
Mariusz Kr...
Metadata: {'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2018-03-05T09:43:57+01:00', 'author': 'agimeno', 'moddate': '2018-03-12T10:24:10-04:00', 'source': '..\\data\\pdf\\test2.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1', 'source_file': 'test2.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2018-03-05T09:43:57+01:00', 'author': 'agimeno', 'moddate': '2018-03-12T10:24:10-04:00', 'source': '..\\data\\pdf\\test2.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1', 'source_file': 'test2.pdf', 'file_type': 'pdf'}, page_content='The EUROCALL Review, Volume 25, No. 2, September 2017 \n \n 18 \nResearch paper \n \nA look at advanced learners’ use of mobile devices for \nEnglish language study: Insights from interview data \nMariusz Kruk \nUniversity of Zielona Gora, Poland \n______________________________________________________________ \nmkruk @ uz.zgora.pl \n  \nAbstract \nThe paper discusses the results of a study which explored advanced learners of English \nengagement with their mobile devices to develop learning experiences that meet their \nneeds and goals as foreign language learners. The data were collected from 20 students \nby means of a semi -structured interv

In [92]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [93]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### Vector Store

In [94]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 224


In [95]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store in the vector database
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 56 texts...


Batches: 100%|██████████| 2/2 [00:00<00:00,  2.72it/s]

Generated embeddings with shape: (56, 384)
Adding 56 documents to vector store...
Successfully added 56 documents to vector store
Total documents in collection: 280


### Retriever Pipeline From VectorStore

In [96]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)
rag_retriever

In [97]:
rag_retriever.retrieve("look at advanced learners")

Retrieving documents for query: 'look at advanced learners'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 408.24it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_4fb29834_2',
  'content': 'of mobile apps available, easy access to the internet) and/or the fact tha t they are \nincreasingly more common among learners make them an important and pot entially \nuseful addition to formal and informal language learning. \nAccording to Benson (2011), there has always been a connection between educational \ntechnologies and learner autonomy to the extent that they have often been intended for \nindependent practice. It should be noted, however, that this link and “future enquiry \nand practice into technology -mediated learner autonomy will need to be increasingly \naligned to the tools, settings, and activities that are of significance to language learners” \n(Reinders & White, 2016, p. 151). Reinders and White (2016) further argue that  as long \nas “the potential range of settings, tools, and experiences is now virtually limitless, \nindividuals need to be increasingly adept at critical adaptive learning in order to benefit',
  'metadata

### RAG Pipeline- VectorDB To LLM Output Generation

In [98]:
import os
from dotenv import load_dotenv
load_dotenv()

print(os.getenv("DeepSeek_API_KEY"))

In [99]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

In [100]:
class DeepSeekLLM:
    def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
        """
        Initialize DeepSeek LLM
    
        Args:
            model_name: DeepSeek model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: DeepSeek API key (or set DEEPSEEK_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("DeepSeek_API_KEY")

        if not self.api_key:
            raise ValueError("DeepSeek API key is required. Set DEEPSEEK_API_KEY environment variable or pass api_key parameter.")

        self.llm = ChatDeepSeek(
            deepseek_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )

        print(f"Initialized DeepSeek LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)
        
        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
    


In [101]:
# Initialize DeepSeek LLM (you'll need to set DEEPSEEK_API_KEY environment variable)
try:
    deepseek_llm = DeepSeekLLM(api_key=os.getenv("DeepSeek_API_KEY"), model_name="gpt-oss-20b")
    print("DeepSeek LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your DEEPSEEK_API_KEY environment variable to use the LLM.")
    deepseek_llm = None

Initialized DeepSeek LLM with model: gpt-oss-20b
DeepSeek LLM initialized successfully!


In [102]:
rag_retriever.retrieve("look at advanced learners")

Retrieving documents for query: 'look at advanced learners'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 493.62it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_4fb29834_2',
  'content': 'of mobile apps available, easy access to the internet) and/or the fact tha t they are \nincreasingly more common among learners make them an important and pot entially \nuseful addition to formal and informal language learning. \nAccording to Benson (2011), there has always been a connection between educational \ntechnologies and learner autonomy to the extent that they have often been intended for \nindependent practice. It should be noted, however, that this link and “future enquiry \nand practice into technology -mediated learner autonomy will need to be increasingly \naligned to the tools, settings, and activities that are of significance to language learners” \n(Reinders & White, 2016, p. 151). Reinders and White (2016) further argue that  as long \nas “the potential range of settings, tools, and experiences is now virtually limitless, \nindividuals need to be increasingly adept at critical adaptive learning in order to benefit',
  'metadata

### Integration Vectordb Context pipeline With LLM output

In [107]:
from langchain_deepseek import ChatDeepSeek
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the DeepSeek LLM (set your DEEPSEEK_API_KEY in environment)
deepseek_api_key = os.getenv("DeepSeek_API_KEY")

# Add the model parameter - DeepSeek's chat model
llm = ChatDeepSeek(
    model="deepseek-chat",  # or "deepseek-coder" for coding tasks
    api_key=deepseek_api_key,
    temperature=0.1,
    max_tokens=1024
)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, llm, top_k=3):
    ## retrieve the context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answer using DeepSeek LLM
    prompt = f"""Use the following context to answer the question concisely.
Context:
{context}

Question: {query}

Answer:"""
    
    # Note: prompt is already formatted, so just pass it directly
    response = llm.invoke([{"role": "user", "content": prompt}])
    return response.content

In [108]:
answer=rag_simple("look at advanced learner",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'look at advanced learner'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 80.48it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Based on the context, an advanced learner needs to be adept at **critical adaptive learning** to benefit from the vast and limitless range of digital tools, settings, and experiences available for language learning.


In [109]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("Hard Negative Mining Technqiues", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'Hard Negative Mining Technqiues'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 126.98it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
Answer: No relevant context found.
Sources: []
Confidence: 0.0
Context Preview: 


In [110]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is attention is all you need", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'what is attention is all you need'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 96.82it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)



Final Answer: No relevant context found.
Summary: The system could not locate any relevant information to address the query. Therefore, no summary can be provided.
History: {'question': 'what is attention is all you need', 'answer': 'No relevant context found.', 'sources': [], 'summary': 'The system could not locate any relevant information to address the query. Therefore, no summary can be provided.'}
